<a href="https://colab.research.google.com/github/brininha/rastreador_aprovados/blob/main/rastreador_aprovados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de rastreador de aprovados
**Desenvolvido por:** InovaTec

Este é o sistema automatizado para conferência de listas de aprovados.

---

### Guia de como usar este rastreador:

**Prepare suas planilhas**

Tenha em mãos o arquivo de **alunos do cursinho** (Excel ou CSV) e a **lista oficial de aprovados** divulgada pela universidade. Para esse protótipo você pode baixar os arquivos com nomes fictícios de exemplo executando o primeiro script que aparece.

**Rode o script**

Clique no botão de iniciar (▶️) que aparecerá no canto esquerdo das células de código abaixo.

**Faça o upload**

O sistema pedirá primeiro a planilha de alunos. Clique em **"Escolher arquivos"** e envie. Em seguida, ele pedirá a lista oficial. Envie o segundo arquivo.

**Analise o resultado**

O sistema exibirá uma tabela com os aprovados encontrados. Um arquivo chamado "Relatorio_Aprovados.xlsx" será gerado automaticamente. Para baixá-lo, clique no ícone de pasta (📁) na barra lateral esquerda deste site.

---

**Para iniciar, execute os scripts abaixo**

In [ ]:
# @title 👇 Clique no ícone abaixo para baixar as planilhas de teste
import pandas as pd
from google.colab import files # biblioteca essencial para o download

# criação dos dados fictícios
dados_alunos = {"Nome": ["Ana Silva", "Carlos Souza", "Maria Oliveira", "João Pedro"]}
dados_fuvest = {"Nome_Oficial": ["ANA SILVA DE SOUZA", "CARLOS SOUZA", "PEDRO ALMEIDA", "MARIA OLIVEIRA"]}

# definindo nomes dos arquivos
nome_arq_alunos = "exemplo_alunos_cpe.xlsx"
nome_arq_oficial = "exemplo_fuvest.xlsx"

# salvando no ambiente temporário do Colab
pd.DataFrame(dados_alunos).to_excel(nome_arq_alunos, index=False)
pd.DataFrame(dados_fuvest).to_excel(nome_arq_oficial, index=False)

print(f"Arquivos '{nome_arq_alunos}' e '{nome_arq_oficial}' criados com sucesso!")
print("Iniciando download automático para o seu computador...")

# forçando o download pelo navegador
files.download(nome_arq_alunos)
files.download(nome_arq_oficial)

Arquivos 'exemplo_alunos_cpe.xlsx' e 'exemplo_fuvest.xlsx' criados com sucesso!
Iniciando download automático para o seu computador...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 👇 Clique no ícone abaixo para iniciar a busca por aprovados
# instalação das bibliotecas
!pip install rapidfuzz pandas openpyxl unidecode tqdm

import pandas as pd
from rapidfuzz import process, fuzz
from unidecode import unidecode
from google.colab import files
from tqdm import tqdm
import io

print("Sistema de rastreador de aprovados")

# função para limpar texto (remove acentos e padroniza)
def normalizar_texto(texto):
    if pd.isna(texto): return ""
    texto = str(texto).upper()
    texto = unidecode(texto) # remove acentos
    return " ".join(texto.split()) # remove espaços duplos

# função de upload
def carregar_arquivo(nome_tipo):
    print(f"\n>> Selecione o arquivo de {nome_tipo} (.csv ou .xlsx):")
    uploaded = files.upload()
    if not uploaded: return None
    nome_arquivo = next(iter(uploaded))
    if nome_arquivo.endswith('.csv'):
        return pd.read_csv(io.BytesIO(uploaded[nome_arquivo]))
    elif nome_arquivo.endswith('.xlsx'):
        return pd.read_excel(io.BytesIO(uploaded[nome_arquivo]))
    else:
        print("Formato inválido!")
        return None

# carregamento
df_alunos = carregar_arquivo("alunos do cursinho")
df_oficial = carregar_arquivo("lista oficial")

if df_alunos is not None and df_oficial is not None:
    print("\nIniciando cruzamento de dados...")

    # identificar automaticamente a coluna de nomes
    col_alunos = df_alunos.select_dtypes(include=['object']).columns[0]
    col_oficial = df_oficial.select_dtypes(include=['object']).columns[0]

    # pré-processamento
    df_alunos['nome_limpo'] = df_alunos[col_alunos].apply(normalizar_texto)
    lista_oficial_limpa = [normalizar_texto(x) for x in df_oficial[col_oficial].tolist()]

    # dicionário para mapear nome limpo -> nome original da lista oficial
    # isso serve para exibir o nome bonito no final, não o limpo
    mapa_nomes_oficial = {normalizar_texto(x): x for x in df_oficial[col_oficial].tolist()}

    resultados = []

    # barra de progresso para acompanhar o processamento
    for nome_limpo_aluno in tqdm(df_alunos['nome_limpo'], desc="Processando alunos"):

        # usei rapidfuzz para extrair o melhor candidato
        # score_cutoff=50 significa que ele ignora qualquer coisa abaixo de 90% de chance
        match = process.extractOne(nome_limpo_aluno, lista_oficial_limpa, scorer=fuzz.token_sort_ratio, score_cutoff=90)

        if match:
            nome_match_limpo, score, indice = match
            nome_original_aluno = df_alunos.loc[df_alunos['nome_limpo'] == nome_limpo_aluno, col_alunos].values[0]
            nome_original_oficial = mapa_nomes_oficial[nome_match_limpo]

            # define status baseado na confiança do algoritmo
            status = "✅ Aprovado" if score >= 98 else "⚠️ Inconclusivo (verificar)"

            resultados.append({
                "Aluno CPE": nome_original_aluno,
                "Nome na lista": nome_original_oficial,
                "Similaridade": f"{score:.1f}%",
                "Status": status
            })

    # exibição
    if resultados:
        df_final = pd.DataFrame(resultados).sort_values(by="Similaridade", ascending=False)
        print(f"\n🎉 SUCESSO! Encontrados {len(resultados)} possíveis aprovados.")
        display(df_final)
        df_final.to_excel("resultado_cruzamento.xlsx", index=False)
    else:
        print("\nNenhum match encontrado com os critérios definidos.")

Sistema de rastreador de aprovados

>> Selecione o arquivo de alunos do cursinho (.csv ou .xlsx):


Saving alunos_cpe.csv to alunos_cpe.csv

>> Selecione o arquivo de lista oficial (.csv ou .xlsx):


Saving lista_unesp_limpa (1).csv to lista_unesp_limpa (1) (2).csv

Iniciando cruzamento de dados...


Processando alunos: 100%|██████████| 147/147 [00:02<00:00, 49.18it/s]


🎉 SUCESSO! Encontrados 23 possíveis aprovados.


,Aluno CPE,Nome na lista,Similaridade,Status
6,Muriel Rodrigues da Silva,MURILO DA SILVA RODRIGUES,96.0%,⚠️ Inconclusivo (verificar)
14,Daniel De Oliveira Souza,DANIEL OLIVEIRA DE SOUSA,95.8%,⚠️ Inconclusivo (verificar)
0,Camilly Eduarda Dos Santos Pereira,CAMILLY EDUARDA SANTOS PEREIRA,93.8%,⚠️ Inconclusivo (verificar)
21,Núbia Oliveira Santos,JULIA OLIVEIRA SANTOS,90.5%,⚠️ Inconclusivo (verificar)
12,Davi Araujo Silva,DAVI ARAUJO SILVA,100.0%,✅ Aprovado
20,Nicolle Venâncio dos Santos,NICOLLE VENANCIO DOS SANTOS,100.0%,✅ Aprovado
19,Mirella de Sousa Berbel,MIRELLA DE SOUSA BERBEL,100.0%,✅ Aprovado
18,Mikaelle Vitoria Vieira De Lima,MIKAELLE VITORIA VIEIRA DE LIMA,100.0%,✅ Aprovado
17,Maria Clara Silva de Oliveira,MARIA CLARA DE OLIVEIRA SILVA,100.0%,✅ Aprovado
16,juliana dos santos silva,JULIANA DOS SANTOS SILVA,100.0%,✅ Aprovado
